# Set Partitioning on Dirac
#### Device: Dirac-1

## Introduction
The set partitioning problem is an optimization problem which selects sets $S_i$ from a collection $S$ such that each member $x\in X=\bigcup_i S_i$ is included in exactly one $S_i$ of the selected sets (see [Operations Research Journal Vol. 17, No. 5](https://doi.org/10.1287/opre.17.5.848)). It has applications in logistics, design, manufacturing and scheduling, among others. In more mathematical literature, the set partitioning problem is sometimes referred to as exact cover.

The set partitioning problem is formulated by creating constraints which specify that only one out of all the sets $S_i$ of which a particular member $s$ is selected. This looks like
$$
\sum_{i,x\in S_i} s_i = 1\quad \forall x\in X
$$
where $s_i\in\{0,1\}$ indicates set $S_i$ is selected. In addition to a constraint for each member, there is an objective function which measures the cost, weight or benefit of selecting certain sets from $S$. The objective function could be any form, but we can solve linear or quadratic objective functions with Dirac. A quadratic objective function has coefficients $c_{ij},c_{i}$ for quadratic and linear terms, respectively. With the variables $s_i$, we have
$$
\sum_j\sum_i c_{ij}s_is_j + \sum_i c_is_i.
$$



## Importance

While the problem of set partitioning may first seem esoteric, the underlying concepts are actually quite natural and often arise in the real world, as do closely related problems. It also has a natural constraint structure that often appears in many problems (eg. one-hot constraints), enforcing that every member of each set is included once and only once. Unlike the two-way one-hot constraints seen in [travelling salesperson](https://quantumcomputinginc.com/learn/tutorials-and-use-cases/traveling-salesperson-on-dirac) or the [quadratic assignment problem](https://quantumcomputinginc.com/learn/tutorials-and-use-cases/quadratic-assignment-on-dirac), the one-hot constraints here do not form a regular structure. However, they do overlap since each set will have one constraint for each element in the set. A natural generalization of set partitioning is set cover, where the goal is to find the minimum weight configuration where each element is included at least once, but may be duplicated. A review of problems related to set partitioning and their applications can be found [here](https://link.springer.com/chapter/10.1007/978-1-4613-0303-9_).


## Applications

One perhaps surprising application to set partitioning is [corporate tax structuring](https://digital.case.edu/islandora/object/ksl%3AweaopeTM0362). The set partitioning constraints originate from the fact that subsidiaries can be combined for tax purposes, but each of subsidiary cannot be contained in more than one conglomerate. Since taxes must be paid on all of them, each has to be contained somewhere. Another application is [airline crew scheduling](https://www.gsb.stanford.edu/faculty-research/working-papers/exact-solution-crew-scheduling-problems-using-set-partitioning-model). In this setting, the sets are different round trips that could be assigned to a type of crew member, and exactly one of which is needed for each leg of a flight. This model straightforwardly extends to similar applications such as bus driver scheduling. The [formation of quarantine groups](https://www.tandfonline.com/doi/full/10.1080/24725854.2023.2192250) that minimize the spread of disease is another application, in this case the sets are the potential groupings, and the members of the sets are individuals being quarantined. It is clearly undesirable to place someone in multiple such groups, but every member has to go somewhere, hence the set partitioning constraint.

## Implementation

In this tutorial, we utilize SetPartitionModel, an eqc-models optimization model, to solve the set partitioning problem as a [quadratic unconstrained binary optimization](https://quantumcomputinginc.com/learn/tutorials-and-use-cases/qubo-on-dirac) (QUBO) problem. The SetPartitionModel class seamlessly handles the constraints and objective function, encoding them into a solver-compatible format without requiring manual penalty construction.

The constraints, defined as ensuring each element in the universal set $X$ is included in exactly one selected subset, are internally represented and enforced by the model. The model also constructs the quadratic objective function from the given weights.

To enforce the constraints in SetPartitionModel, the model applies a penalty function $P(s)$, defined as:
$$
P(s)=s^T(A^TA)s-(2b^TA)^Ts+b^Tb,
$$
where $A$ is a binary matrix indicating subset membership and $b$ is a vector of ones enforcing the "exactly one subset" constraint. When all constraints are met, $P(s)=0$ and when any constraint is violated, $P(s)>0$.

Henceforth, the total function to be optimized, combining the quadratic objective and the constraint penalty, is:
$$
Q(s)=\sum_j\sum_i c_{ij}s_is_j + \sum_i c_is_i + \alpha P(s),
$$
where $c_{ij}$ and $c_i$ represent the quadratic and linear terms of the objective and $\alpha$ is a penalty multiplier ensuring infeasible solutions are penalized more heavily than feasible ones. This formulation allows the SetPartitionModel to encode both the constraints and the objective function into a single Hamiltonian for optimization.

**Solving with Dirac1CloudSolver**

The SetPartitionModel constructs the Hamiltonian $Q(s)$ and integrates seamlessly with the Dirac1CloudSolver, QCi's entropy quantum computing solver, capable of handling optimization problems efficiently. The solver minimizes $Q(s)$ to find an optimal or near-optimal solution, ensuring the constraints are respected.

## Demonstration

In [1]:
import numpy as np
from eqc_models.assignment.setpartition import SetPartitionModel
from eqc_models.solvers import Dirac1CloudSolver

### Creating the Data
$S$ is a collection of 9 different sets. The members of the sets are the letters A through F. $W$ are the weights of each subset $S_i$.

In [2]:
np.random.seed(21)
S = [{"A", "B", "C"}, {"D", "E", "F"}, {"A", "F"}, {"B", "E"}, {"C", "D"}, {"A"},
     {"B"}, {"C", "D", "E"}, {"B", "C"}]
W = [100 * np.random.random() for S_i in S]

Initialize the `SetPartitionModel`

In [3]:
model = SetPartitionModel(subsets=S, weights=W)
model.upper_bound = np.ones(len(S))
model.penalty_multiplier = 20

Create a connection to the REST API using `Dirac1CloudSolver`.

In [5]:
token = "your_token"
api_url = "https://api.qci-prod.com"
solver = Dirac1CloudSolver(api_token=token, url=api_url)

Solve the problem using `solve` of `Dirac1CloudSolver`

In [6]:
response = solver.solve(model, num_samples=2)

2025-01-09 10:46:31 - Dirac allocation balance = 0 s (unmetered)
2025-01-09 10:46:32 - Job submitted: job_id='677fef588dd54d8a9ea6064b'
2025-01-09 10:46:32 - QUEUED
2025-01-09 10:46:35 - RUNNING
2025-01-09 10:46:57 - COMPLETED
2025-01-09 10:47:00 - Dirac allocation balance = 0 s (unmetered)


The iteration over the samples tests if each sample selected are members of $X$ (defined in `get_union` below) exactly once. 100% set coverage indicates that all members were included and a partition was found where no member is included in multiple sets simultaneously.

In [8]:
def get_union(S):
    X = set()
    for S_i in S:
        X = X.union(S_i)
    return X

def check_result(results, objective=None):
    samples = results["solutions"]
    energies = results["energies"]
    counts = results["counts"]
    X = get_union(S)
    for j, sample in enumerate(samples):
        print("QUBO value:", energies[j])
        print("Times sample found", counts[j])
        sample = np.array(sample)
        if objective is not None:
            print("Objective Function value:", sample.T@objective@sample)
        print("Selected Sets")
        testX = set()
        members = []
        for i in range(len(S)):
            if sample[i] == 1:
                print(f"S_{i}", end=" ")
                testX = testX.union(S[i])
                members.extend(S[i])
        print()
        print(f"Set coverage {100*len(testX)/len(X)}%")
        print(f"Partition found: {len(testX)==len(members)}")

def get_results(response):
    if "results" in response and response["results"] is not None:
        results = response["results"]
    else:
        if "job_info" in response and "job_result" in response["job_info"]:
            details = response["job_info"]["job_result"]
        else:
            details = None
        raise RuntimeError(f"Execution failed. See details: {details}")
    return results

In [9]:
results = get_results(response)
check_result(results)
# save the response
penalty_only_response = response

QUBO value: -86.21652579307556
Times sample found 2
Selected Sets
S_0 S_1 
Set coverage 100.0%
Partition found: True


## Conclusion

In this tutorial, we have demonstrated how to solve the set partitioning problem on our Dirac-1 device. The `SetPartitionModel` encapsulates the complexities of constraint handling and Hamiltonian construction. By combining the quadratic objective with a robust penalty formulation, where $P(s)$ ensures constraints are enforced and $Q(s)$ encodes the total optimization function, the model enables solvers like `Dirac1CloudSolver` to deliver efficient and accurate solutions to combinatorial optimization problems.